# DynScene: Scalable Generation of Dynamic Robotic Manipulation Scenes for Embodied AI
## 文献出处及作者
* 会议：CVPR 2025
* 作者：Sangmin Lee*, Sungyong Park*, Heewon Kim 崇实大学
## 文献主要内容
* 提出了DynScene，基于扩散模型的统一框架，能从自然指令中联合生成动态的机器人操作场景，该场景包含静态初始环境和对应的机器人动作序列
* 核心组件包括：<br>
**静态场景生成**：使用条件扩散模型，根据文本指令生成机器人、目标物体的初始位姿和状态<br>
**布局采样**：从数据库中采样合理的房间布局，以避免碰撞和确保目标可达<br>
**四元数量化**：对物体朝向进行离散化，防止因微小角度误差导致物体因重力倾倒<br>
**残差动作生成**：使用另一扩散模型，以静态场景为条件，生成残差坐标表示的动作序列，增强了不同场景的泛化能力<br>
**动作增强**：利用残差表示，可将不同静态场景与动作序列组合，大幅增加数据多样性<br>
**物理仿真验证**：所有场景均在lsaac Sim中进行验证，仅保留成功执行的场景用于训练，确保数据质量
## 关键贡献
1. 提出了首个统一框架，实现了从文本指令到动态场景（静态环境+机器人动作） 的端到端生成，解决了场景与动作割裂的问题。
2. 引入了新颖的残差动作表示与增强机制，使生成的动作能更好地泛化到不同的空间配置和目标状态，显著提升了数据多样性。
3. 设计了一系列基于扩散模型的生成与优化技术（如布局采样、四元数量化），共同保证了生成场景的物理合理性与任务可行性。
4. 通过充分的实验验证，证明了DynScene在生成效率、成功率、动作多样性方面均显著优于人类专家数据和现有方法，并能作为高质量训练数据，实质性提升下游机器人操作智能体的性能。
## DynScene框架
### 动态场景的数据表示
1. 动态场景的定义：<br>
动态场景是一个二元组(s,a)，由文本指令驱动。<br>
**s**：静态场景，代表机器人和所有物体的初始位置<br>
**a**：动作序列，代表机器人执行的一系列操作
2. 静态场景数据（使用绝对坐标）静态场景s被定义为一个包含所有必要信息的向量：
$$s=[o,r,z]$$
* ***o:目标物体的信息<br>***
$p^{obj},q^{obj}$：物体的3D位置和四元数朝向<br>
c：物体类别（如“杯子”、“抽屉”）<br>
v：物体尺寸<br>
f：形状编码（点云嵌入向量）<br>
$s^{init},s^{goal}$ ：任务的初始状态和目标状态（如抽屉的开合程度）。这是支持连续状态任务的关键
* ***r：机器人的初始状态***<br>
$p^{base},q^{base}$：机器人基座的位置和朝向<br>
$p^{ee}_0,q^{ee}_0$：末端执行器（手）的初始位置和朝向<br>
$g_0$：夹爪的初始状态（“打开”或“关闭”）
* ***z：房间布局，包括障碍物，家具等环境结构***
3.  动作数据（使用残差坐标）
动作序列a被划分为K个步骤，每一步动作$a_k$定义为：
$$a_k=[\Delta p^{ee}_k,\Delta q^{ee}_k,g_k]$$
$\Delta p^{ee}_k=p^{ee}_k-p^{ee}_{k-1}$：末端执行器位置的残差变化<br>
$\Delta q^{ee}_k=q^{ee}_k-q^{ee}_{k-1}$：末端执行器朝向的残差变化<br>
$g_k$：该步骤的夹爪状态

### 静态场景的生成与优化
* 根据文本指令y生成一个合理，稳定，可执行的初始场景s
1. 条件扩散模型
* 前向过程：
$$q(s_t|s_0)=N(s_t;\sqrt{\overline{\alpha}_t}s_0,(1-\overline{\alpha}_t)I)$$
$s_0$：原始静态场景数据。这是一个向量，包含了机器人位姿、物体位置、朝向等所有静态场景信息
$t$：时间步。表示噪声添加的步骤
$s_t$：在时间步$t$的噪声场景数据。这是原始数据与噪声的混合体。
$\alpha_t$：噪声调度参数。这是一个预先定义好的、随时间t递减的序列，满足$0<\alpha_t<1$，它控制了每一步添加的噪声量
$\overline{\alpha}_t$：累积噪声调度参数。定义为$\overline{\alpha}_t=\prod^{t}_{i=1}\alpha_i$。由于每个 $\alpha_i$都小于1，所以$\overline{\alpha}_t$随着$t$增大而逐渐趋于0
$\overline{\alpha}_t s_0$：噪声数据的均值。随着$t$增大，$\alpha_t\rightarrow0$，均值也趋近于0
$(1-\alpha_t)$噪声数据的方差。随着$t$增大，$\alpha_t\rightarrow0$，方差也趋近于0
$I$：单位矩阵，表示噪声在各个维度上是独立同分布的
* 反向过程，从噪声中重建数据：
$$p_\theta(s_{t-1}|s_t,y)=N(s_{t-1};\mu_\theta(s_t,t;y),\sigma^2_tI)$$
$p_\theta(s_{t-1}|s_t,y)$：条件概率分布。表示在给定当前噪声数据$s_t$和文本条件$y$的情况下，预测前一个时间步$s_{t-1}$的分布
$\mu_\theta(s_t,t;y)$：由神经网络预测的均值。这是反向过程的核心，神经网络需要根据当前噪声、时间步和文本指令，预测出"更清晰一点"的数据应该是什么样子。
$\sigma^2_t$：方差。通常是一个与时间步$t$相关的固定值，不需要学习。
$\theta$：神经网络参数
* 训练目标的实现，噪声预测目标：
$$L(\theta)_{text}=E_{s_0,\epsilon,t}[||\epsilon-\epsilon_\theta(s_t,t;y)||^2]$$